In [1]:
import pandas as pd
import numpy as np
import datetime as DT
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score
import random

# Comenzamos con los datos de genero y edad

Concatenamos los datos disponibles en un solo DF

In [2]:
concatAge = pd.concat([pd.read_csv('data/datos_navent_fiuba/fiuba_2_postulantes_genero_y_edad.csv'),
                       pd.read_csv('data/entrega6/fiuba_2_postulantes_genero_y_edad.csv'),
                       pd.read_csv('data/Fiuba desde 15 Abril/fiuba_2_postulantes_genero_y_edad.csv')]
                    )
concatAge.head()

,idpostulante,fechanacimiento,sexo
0,NM5M,1970-12-03,FEM
1,5awk,1962-12-04,FEM
2,ZaO5,1978-08-10,FEM
3,NdJl,1969-05-09,MASC
4,eo2p,1981-02-16,MASC


Eliminamos los datos duplicados, quedandonos con el ultimo disponible. Ademas calculamos
la edad de cada usuario en base a su fecha de nacimiento. 

In [3]:
concatAge.drop_duplicates('idpostulante',keep = 'last', inplace = True)
concatAge.reset_index(inplace = True)
concatAge.drop(columns = ['index'],inplace = True)
concatAge.rename(columns = {'fechanacimiento':'edad'}, inplace = True)


# hoy : 30/5/2018
hoy = pd.Timestamp(DT.datetime.now())
concatAge['edad'] = pd.to_datetime(concatAge['edad'],errors = 'coerce')
concatAge['edad'] = (hoy - concatAge['edad']).astype('<m8[Y]')

In [4]:
concatAge['sexo'].value_counts()

FEM           251205
MASC          227853
NO_DECLARA     25342
0.0                7
Name: sexo, dtype: int64

Transformamos la columnas del genero de los usuarios al tipo booleano, indicando si son o no
usuarios masculinos. A los que no declararon sexo se les otorga un sexo random (hay que mejorar esto). Por suerte es una cantidad acotada (5%)

In [5]:
def sexoABool(sexo):
    if (sexo == 'MASC'):
        return True
    if (sexo == 'FEM'):
        return False
    return (bool(random.getrandbits(1)))

In [6]:
concatAge['sexo'] = concatAge['sexo'].apply(sexoABool)
concatAge.rename(columns = {'sexo':'sexo_masculino'}, inplace = True)
concatAge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 504407 entries, 0 to 504406
Data columns (total 3 columns):
idpostulante      504407 non-null object
edad              478311 non-null float64
sexo_masculino    504407 non-null bool
dtypes: bool(1), float64(1), object(1)
memory usage: 8.2+ MB


Con la edad sucede algo parecido en cuanto a los datos faltantes. Los rellenamos con numeros
cuya probabilidad corresponden a una distribucion exponencial, manteniendo la media original.

In [7]:
media = concatAge['edad'].mean()
concatAge['edad'].fillna(int(random.expovariate(1/media)),inplace = True)

In [8]:
def recalcularEdad(edad):
    if ((edad < 18) or (edad > 75)):
        nueva_edad = int(random.expovariate(1/media))
        while ((nueva_edad < 18) or (nueva_edad>75)):
            nueva_edad = int(random.expovariate(1/media))
        return nueva_edad
    return edad

In [9]:
concatAge['edad'] = concatAge['edad'].apply(recalcularEdad)

In [10]:
concatAge.edad.value_counts()

26.0    27178
25.0    26701
27.0    26661
24.0    26287
23.0    25801
22.0    24261
28.0    23631
29.0    22708
21.0    22295
30.0    20671
31.0    19678
20.0    18916
32.0    17697
33.0    15792
19.0    14670
34.0    14620
35.0    13660
36.0    13047
37.0    12195
38.0    11263
39.0    10541
40.0     9480
41.0     8490
18.0     8180
42.0     7727
43.0     7036
44.0     6019
45.0     5350
46.0     4961
47.0     4465
48.0     4043
49.0     3438
50.0     3141
51.0     2804
52.0     2566
53.0     2360
54.0     2080
55.0     1935
56.0     1824
57.0     1620
58.0     1360
59.0     1136
60.0      929
61.0      857
62.0      665
63.0      549
64.0      464
65.0      368
66.0      335
67.0      305
68.0      279
70.0      243
69.0      225
71.0      216
73.0      192
72.0      179
75.0      166
74.0      147
Name: edad, dtype: int64

# Pasamos a los datos sobre la educacion

Concatenamos todos los datos disponibles sobre la educacion de los usuarios

In [11]:
concatEduc = pd.concat([pd.read_csv('data/datos_navent_fiuba/fiuba_1_postulantes_educacion.csv'),
                        pd.read_csv('data/entrega6/fiuba_1_postulantes_educacion.csv'),
                        pd.read_csv('data/Fiuba desde 15 Abril/fiuba_1_postulantes_educacion.csv')]
                      )
concatEduc.head()

,idpostulante,nombre,estado
0,NdJl,Posgrado,En Curso
1,8BkL,Universitario,En Curso
2,1d2B,Universitario,En Curso
3,NPBx,Universitario,En Curso
4,NPBx,Master,En Curso


In [12]:
concatEduc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1103193 entries, 0 to 397508
Data columns (total 3 columns):
idpostulante    1103193 non-null object
nombre          1103193 non-null object
estado          1103193 non-null object
dtypes: object(3)
memory usage: 33.7+ MB


In [13]:
concatEduc['nombre'].value_counts()

Universitario        404619
Secundario           382927
Terciario/Técnico    180091
Otro                  84674
Posgrado              33484
Master                16436
Doctorado               962
Name: nombre, dtype: int64

Agregamos columnas que indiquen si el usuario está graduado en educacion de determinado
nivel

In [14]:
def estudioSuperior(nivel):
    if ((nivel=='Posgrado') or (nivel =='Doctorado') or (nivel=='Master')):
        return 1
    else:
        return 0

In [15]:
graduados = (concatEduc['estado'].astype(str)== 'Graduado').astype(int)

universitarios = (concatEduc['nombre'].astype(str) == 'Universitario').astype(int)

secundario = (concatEduc['nombre'].astype(str) == 'Secundario').astype(int)

mayor_a_universitario = concatEduc['nombre'].apply(estudioSuperior)

In [16]:
concatEduc['titulo_universitario'] = universitarios*graduados
concatEduc['titulo_secundario'] = secundario*graduados
concatEduc['titulo_superior'] = mayor_a_universitario * graduados

In [17]:
titulos = concatEduc.groupby('idpostulante').agg({'titulo_universitario':'max',
                                                   'titulo_secundario':'max',
                                                   'titulo_superior':'max'})
titulos.reset_index(inplace = True)

titulos.head()

,idpostulante,titulo_universitario,titulo_secundario,titulo_superior
0,0z5Dmrd,1,0,0
1,0z5JW1r,0,1,0
2,0z5VvGv,0,1,0
3,0zB01pE,0,0,0
4,0zB026d,0,1,0


Aqui creamos un nuevo DF llamado 'usuarios', donde cada fila corresponde a un usuario y 
posee las columnas que corresponden a los datos sobre su educacion, edad y genero

In [18]:
usuarios = concatAge.merge(titulos, on = 'idpostulante', how = 'left')
usuarios.head()

,idpostulante,edad,sexo_masculino,titulo_universitario,titulo_secundario,titulo_superior
0,eo2p,37.0,True,1.0,1.0,1.0
1,1d2B,42.0,True,0.0,0.0,0.0
2,EBO0,44.0,False,1.0,0.0,1.0
3,a6MKW,43.0,True,1.0,0.0,1.0
4,6MWd4,43.0,True,1.0,0.0,0.0


convertimos a booleano algunas columnas para reducir tamaño

In [19]:
usuarios['titulo_universitario'] = usuarios['titulo_universitario'].astype(bool)
usuarios['titulo_secundario'] = usuarios['titulo_secundario'].astype(bool)
usuarios['titulo_superior'] = usuarios['titulo_superior'].astype(bool)

In [20]:
usuarios.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 504407 entries, 0 to 504406
Data columns (total 6 columns):
idpostulante            504407 non-null object
edad                    504407 non-null float64
sexo_masculino          504407 non-null bool
titulo_universitario    504407 non-null bool
titulo_secundario       504407 non-null bool
titulo_superior         504407 non-null bool
dtypes: bool(4), float64(1), object(1)
memory usage: 13.5+ MB


# Pasamos a los datos sobre los avisos

Concatenamos toda la informacion sobre avisos

In [21]:
avisos = pd.concat([pd.read_csv('data/datos_navent_fiuba/fiuba_6_avisos_detalle.csv'),
                    pd.read_csv('data/entrega6/fiuba_6_avisos_detalle.csv'),
                    pd.read_csv('data/Fiuba desde 15 Abril/fiuba_6_avisos_detalle.csv'),
                    pd.read_csv('data/fiuba_6_avisos_detalle_missing_nivel_laboral.csv')])


avisos.head()

,idaviso,idpais,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,8725750,1,VENDEDOR/A PROVINCIA DE SANTA FE,<p><strong><strong>Empresa:</strong></strong> ...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Comercial,VENTOR
1,17903700,1,Enfermeras,<p>Solicitamos para importante cadena de farma...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Salud,Farmacias Central Oeste
2,1000150677,1,Chofer de taxi,<p>TE GUSTA MANEJAR? QUERES GANAR PLATA HACIEN...,Capital Federal,NaN,Empedrado 2336,Full-time,Senior / Semi-Senior,Transporte,FAMITAX SRL
3,1000610287,1,CHOFER DE CAMIONETA BAHIA BLANCA - PUNTA ALTA,<p><strong>Somos una empresa multinacional que...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Transporte,Wurth Argentina S.A
4,1000872556,1,Operarios de Planta - Rubro Electrodomésticos,<p><strong>OPERARIOS DE PLANTA</strong></p><p>...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Producción,ELECTRO OUTLET SRL


In [22]:
avisos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45969 entries, 0 to 337
Data columns (total 11 columns):
idaviso                 45969 non-null int64
idpais                  45969 non-null int64
titulo                  45969 non-null object
descripcion             45969 non-null object
nombre_zona             45969 non-null object
ciudad                  225 non-null object
mapacalle               3444 non-null object
tipo_de_trabajo         45969 non-null object
nivel_laboral           45634 non-null object
nombre_area             45969 non-null object
denominacion_empresa    45955 non-null object
dtypes: int64(2), object(9)
memory usage: 4.2+ MB


Eliminamos los datos repetidos, quedándonos con el último disponible

In [23]:
avisos.drop_duplicates('idaviso', keep = 'last', inplace = True)
avisos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25288 entries, 4 to 337
Data columns (total 11 columns):
idaviso                 25288 non-null int64
idpais                  25288 non-null int64
titulo                  25288 non-null object
descripcion             25288 non-null object
nombre_zona             25288 non-null object
ciudad                  160 non-null object
mapacalle               2070 non-null object
tipo_de_trabajo         25288 non-null object
nivel_laboral           24953 non-null object
nombre_area             25288 non-null object
denominacion_empresa    25281 non-null object
dtypes: int64(2), object(9)
memory usage: 2.3+ MB


In [24]:
avisos.idpais.value_counts()

1    25288
Name: idpais, dtype: int64

Eliminamos las columnas cuyos elementos son casi todos nulos, o cuyos datos son todos iguales

In [25]:
avisos.drop(columns = ['mapacalle','ciudad','idpais'], inplace = True)

In [26]:
avisos['tipo_de_trabajo'].value_counts()

Full-time          22831
Part-time           1746
Teletrabajo          248
Por Horas            125
Pasantia             119
Temporario            96
Por Contrato          88
Fines de Semana       28
Primer empleo          6
Voluntario             1
Name: tipo_de_trabajo, dtype: int64

Vemos que la mayoria de los tipos de trabajo son full o part-time. Creamos dos columnas que nos indiquen si el trabajo es Full-time y si es Part-time

In [27]:
avisos['Full-time'] = (avisos['tipo_de_trabajo'].astype(str) == 'Full-time').astype(bool)
avisos['Part-time'] = (avisos['tipo_de_trabajo'].astype(str) == 'Part-time').astype(bool)

In [28]:
avisos['nombre_zona'].value_counts()

Gran Buenos Aires              23017
Capital Federal                 2183
Buenos Aires (fuera de GBA)       57
GBA Oeste                         10
La Plata                           4
Mendoza                            3
Ciudad de Mendoza                  3
Cordoba                            2
Rosario                            2
Tucuman                            1
Corrientes                         1
Santa Cruz                         1
Santa Fe                           1
Neuquen                            1
San Juan                           1
Catamarca                          1
Name: nombre_zona, dtype: int64

Casi el 100% de los avisos son de GBA o capital federal. Creamos una columna booleanaque indique si el trabajo es en GBA

In [29]:
avisos.rename(columns = {'nombre_zona':'GBA'},inplace = True)

In [30]:
avisos['GBA'] = (avisos['GBA'].astype(str) == 'Gran Buenos Aires').astype(bool)
avisos.head(10)

,idaviso,titulo,descripcion,GBA,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,Full-time,Part-time
4,1000872556,Operarios de Planta - Rubro Electrodomésticos,<p><strong>OPERARIOS DE PLANTA</strong></p><p>...,True,Full-time,Senior / Semi-Senior,Producción,ELECTRO OUTLET SRL,True,False
8,9240880,"Productores Asesores Independiente, para venta...",Agente\r\noficial Selecciona:</span></strong><...,False,Full-time,Jefe / Supervisor / Responsable,Comercial,Agencia Oficial Alejandro Arizaga,True,False
19,1110185164,Administrativo de Recepción,<p>Centro Médico Accord se encuentra en la bús...,True,Full-time,Junior,Administración,Unión Personal,True,False
21,1110513885,Distribuidor domiciliario con moto (SAN MIGUEL),<p>Importante correo privado ubicado en SAN MI...,True,Full-time,Junior,Distribución,Asoko Tempo SA,True,False
32,1111034024,Vendedores para venta de medicina pre paga y o...,<p> </p><p><strong>Comercializadora incorpora ...,True,Full-time,Senior / Semi-Senior,Comercial,JELS SRL,True,False
35,1111101289,PM. Zona Norte / Pacheco,<p>Acciona IT se encuentra en la búsqueda de P...,True,Full-time,Senior / Semi-Senior,Liderazgo de Proyecto,ACCIONA IT,True,False
38,1111109704,"Supervisor de obra civil, eléctrico/ instrumen...",<p>Buscamos para importante empresa petrolera ...,True,Full-time,Senior / Semi-Senior,Construcción,Hahn Solarz S.R.L.,True,False
45,1111172357,Empleado Administrativo para Tramites de Habil...,"<p style=""""><strong><em><span style="""">Brujula...",True,Full-time,Senior / Semi-Senior,Administración,Brujula SA,True,False
46,1111174081,Gestor de Cobranzas - Telecobrador,<p><strong>En MAS ACTIVOS BPO te estamos esper...,False,Part-time,Senior / Semi-Senior,Call Center,MAS ACTIVOS S.A.,False,True
47,1111235995,Analista de Recursos Humanos - Quilmes (Plazo...,<p>Cervecería y Maltería Quilmes es una de las...,True,Full-time,Senior / Semi-Senior,Recursos Humanos,AB InBev - Cervecería y Maltería Quilmes,True,False


La columna de tipo de trabajo ya no es relevante, la dropeamos.

In [31]:
avisos.drop(columns = 'tipo_de_trabajo', inplace = True)

In [32]:
avisos['nivel_laboral'].value_counts()

Senior / Semi-Senior                    16975
Junior                                   4152
Otro                                     1977
Jefe / Supervisor / Responsable          1527
Gerencia / Alta Gerencia / Dirección      322
Name: nivel_laboral, dtype: int64

Transformamos la columna de niuvel laborar en 5 features que indiquen si el puesto de trabajo
corresponde o no a determinado nivel

In [33]:
avisos.rename(columns = {'nivel_laboral':'nivel'}, inplace = True)


In [34]:
avisos['senior'] = (avisos['nivel'].astype(str) == 'Senior / Semi-Senior').astype(bool)
avisos['junior'] = (avisos['nivel'].astype(str) == 'Junior').astype(bool)
avisos['otro'] = (avisos['nivel'].astype(str) == 'Otro').astype(bool)
avisos['Jefe'] = (avisos['nivel'].astype(str) == 'Jefe / Supervisor / Responsable').astype(bool)
avisos['Gerencia'] = (avisos['nivel'].astype(str) == 'Gerencia')

In [35]:
avisos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25288 entries, 4 to 337
Data columns (total 14 columns):
idaviso                 25288 non-null int64
titulo                  25288 non-null object
descripcion             25288 non-null object
GBA                     25288 non-null bool
nivel                   24953 non-null object
nombre_area             25288 non-null object
denominacion_empresa    25281 non-null object
Full-time               25288 non-null bool
Part-time               25288 non-null bool
senior                  25288 non-null bool
junior                  25288 non-null bool
otro                    25288 non-null bool
Jefe                    25288 non-null bool
Gerencia                25288 non-null bool
dtypes: bool(8), int64(1), object(5)
memory usage: 1.5+ MB


In [36]:
avisos['nombre_area'].value_counts()

Ventas                                       3153
Comercial                                    1780
Administración                               1708
Producción                                   1536
Programación                                  958
Contabilidad                                  783
Tecnologia / Sistemas                         758
Atención al Cliente                           657
Mantenimiento                                 537
Recursos Humanos                              423
Logística                                     400
Gastronomia                                   394
Call Center                                   391
Soporte Técnico                               382
Almacén / Depósito / Expedición               379
Oficios y Profesiones                         377
Marketing                                     321
Recepcionista                                 306
Mantenimiento y Limpieza                      295
Compras                                       287


Agregamos features que indiquen si el aviso corresponde a una determinada area laboral, para
las N areas laborales que querramos

In [37]:
top_areas = avisos['nombre_area'].value_counts().nlargest(18).index
for area in top_areas:
    avisos[area] = (avisos['nombre_area'] == area).astype(bool)
    
avisos.head()

,idaviso,titulo,descripcion,GBA,nivel,nombre_area,denominacion_empresa,Full-time,Part-time,senior,...,Mantenimiento,Recursos Humanos,Logística,Gastronomia,Call Center,Soporte Técnico,Almacén / Depósito / Expedición,Oficios y Profesiones,Marketing,Recepcionista
4,1000872556,Operarios de Planta - Rubro Electrodomésticos,<p><strong>OPERARIOS DE PLANTA</strong></p><p>...,True,Senior / Semi-Senior,Producción,ELECTRO OUTLET SRL,True,False,True,...,False,False,False,False,False,False,False,False,False,False
8,9240880,"Productores Asesores Independiente, para venta...",Agente\r\noficial Selecciona:</span></strong><...,False,Jefe / Supervisor / Responsable,Comercial,Agencia Oficial Alejandro Arizaga,True,False,False,...,False,False,False,False,False,False,False,False,False,False
19,1110185164,Administrativo de Recepción,<p>Centro Médico Accord se encuentra en la bús...,True,Junior,Administración,Unión Personal,True,False,False,...,False,False,False,False,False,False,False,False,False,False
21,1110513885,Distribuidor domiciliario con moto (SAN MIGUEL),<p>Importante correo privado ubicado en SAN MI...,True,Junior,Distribución,Asoko Tempo SA,True,False,False,...,False,False,False,False,False,False,False,False,False,False
32,1111034024,Vendedores para venta de medicina pre paga y o...,<p> </p><p><strong>Comercializadora incorpora ...,True,Senior / Semi-Senior,Comercial,JELS SRL,True,False,True,...,False,False,False,False,False,False,False,False,False,False


Dropeamos algunas columnas que ya no nos interesan mas

In [38]:
avisos.drop(columns = ['titulo','descripcion','denominacion_empresa','nivel'],
            inplace = True)

In [39]:
avisos.head()

,idaviso,GBA,nombre_area,Full-time,Part-time,senior,junior,otro,Jefe,Gerencia,...,Mantenimiento,Recursos Humanos,Logística,Gastronomia,Call Center,Soporte Técnico,Almacén / Depósito / Expedición,Oficios y Profesiones,Marketing,Recepcionista
4,1000872556,True,Producción,True,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8,9240880,False,Comercial,True,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
19,1110185164,True,Administración,True,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
21,1110513885,True,Distribución,True,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
32,1111034024,True,Comercial,True,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [40]:
avisos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25288 entries, 4 to 337
Data columns (total 28 columns):
idaviso                            25288 non-null int64
GBA                                25288 non-null bool
nombre_area                        25288 non-null object
Full-time                          25288 non-null bool
Part-time                          25288 non-null bool
senior                             25288 non-null bool
junior                             25288 non-null bool
otro                               25288 non-null bool
Jefe                               25288 non-null bool
Gerencia                           25288 non-null bool
Ventas                             25288 non-null bool
Comercial                          25288 non-null bool
Administración                     25288 non-null bool
Producción                         25288 non-null bool
Programación                       25288 non-null bool
Contabilidad                       25288 non-null bool
Tecnologia

# Vistas

Concatenamos todos los datos de vistas en un DF

In [41]:
concatVistas = pd.concat([pd.read_csv('data/datos_navent_fiuba/fiuba_3_vistas.csv'),
                          pd.read_csv('data/entrega6/fiuba_3_vistas.csv'),
                          pd.read_csv('data/Fiuba desde 15 Abril/fiuba_3_vistas.csv')])

concatVistas.head()

,idAviso,timestamp,idpostulante
0,1111780242,2018-02-23T13:38:13.187-0500,YjVJQ6Z
1,1112263876,2018-02-23T13:38:14.296-0500,BmVpYoR
2,1112327963,2018-02-23T13:38:14.329-0500,wVkBzZd
3,1112318643,2018-02-23T13:38:17.921-0500,OqmP9pv
4,1111903673,2018-02-23T13:38:18.973-0500,DrpbXDP


In [42]:
concatVistas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18368813 entries, 0 to 11648229
Data columns (total 3 columns):
idAviso         int64
timestamp       object
idpostulante    object
dtypes: int64(1), object(2)
memory usage: 560.6+ MB


En vez guardar un registro de cada visita realizada, guardamos la cantidad de visitas 
que realizo cada postulante a cada aviso que visitó

In [43]:
concatVistas['visitas'] = 1
concatVistas.rename(columns = {'idAviso':'idaviso'}, inplace = True)

In [44]:
concatVistas = concatVistas.groupby(['idpostulante','idaviso'])\
                           .agg({'visitas':'sum'})\
                           .reset_index()

In [45]:
concatVistas.head()

,idpostulante,idaviso,visitas
0,0002q,1789742,4
1,0002q,1807692,6
2,0002q,1808090,5
3,0005E,1744005,3
4,0005E,1785430,2


In [46]:
concatVistas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8597915 entries, 0 to 8597914
Data columns (total 3 columns):
idpostulante    object
idaviso         int64
visitas         int64
dtypes: int64(2), object(1)
memory usage: 196.8+ MB


# Pasamos a las postulaciones

Concatenamos los datos de postulaciones disponibles

In [47]:
postulaciones = pd.concat([pd.read_csv('data/datos_navent_fiuba/fiuba_4_postulaciones.csv'),
                           pd.read_csv('data/entrega6/fiuba_4_postulaciones.csv')])
postulaciones.head()

,idaviso,idpostulante,fechapostulacion
0,1112257047,NM5M,2018-01-15 16:22:34
1,1111920714,NM5M,2018-02-06 09:04:50
2,1112346945,NM5M,2018-02-22 09:04:47
3,1112345547,NM5M,2018-02-22 09:04:59
4,1112237522,5awk,2018-01-25 18:55:03


Dropeamos la columna de fecha/horario ya que no nos interesa

In [48]:
postulaciones.drop(columns = 'fechapostulacion', inplace = True)

In [49]:
postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8311264 entries, 0 to 4909640
Data columns (total 2 columns):
idaviso         int64
idpostulante    object
dtypes: int64(1), object(1)
memory usage: 190.2+ MB


Eliminamos las postulaciones duplicadas

In [50]:
postulaciones.drop_duplicates(subset = ['idaviso','idpostulante'],
                              keep = 'first',
                              inplace = True)
postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6603752 entries, 0 to 4909640
Data columns (total 2 columns):
idaviso         int64
idpostulante    object
dtypes: int64(1), object(1)
memory usage: 151.1+ MB


### Aca agrego features a los usuarios de acuerdo a las areas a las que postularon

In [51]:
postulaciones = postulaciones.merge(avisos[['idaviso','nombre_area']],
                                    on  = 'idaviso',
                                    how = 'left')

In [52]:
postulaciones.head()

,idaviso,idpostulante,nombre_area
0,1112257047,NM5M,Atención al Cliente
1,1111920714,NM5M,Telemarketing
2,1112346945,NM5M,Telemarketing
3,1112345547,NM5M,Telemarketing
4,1112237522,5awk,Contabilidad


Listamos(manteniendo la multiplicidad), las áreas a las que se postulo cada usuarios

In [53]:
areas_por_postulante = postulaciones.groupby('idpostulante')\
                .agg({'nombre_area': lambda x: list(x),
                      'idaviso':'count'})

areas_por_postulante.head()

,nombre_area,idaviso
idpostulante,,
0z5Dmrd,"[Recursos Humanos, Recursos Humanos]",2
0z5JW1r,"[Almacén / Depósito / Expedición, Comercial, D...",7
0z5VvGv,"[Ventas, Ventas, Atención al Cliente, Atención...",50
0zB01pE,"[Administracion de Seguros, Administracion de ...",3
0zB026d,"[Producción, Producción, Producción, Producció...",10


In [54]:
areas_por_postulante.reset_index(inplace = True)
areas_por_postulante.rename(columns = {'nombre_area':'areas', 'idaviso':'cantidad_postulaciones'},
                            inplace = True)

areas_por_postulante.head()

,idpostulante,areas,cantidad_postulaciones
0,0z5Dmrd,"[Recursos Humanos, Recursos Humanos]",2
1,0z5JW1r,"[Almacén / Depósito / Expedición, Comercial, D...",7
2,0z5VvGv,"[Ventas, Ventas, Atención al Cliente, Atención...",50
3,0zB01pE,"[Administracion de Seguros, Administracion de ...",3
4,0zB026d,"[Producción, Producción, Producción, Producció...",10


In [55]:
areas_por_postulante.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 348883 entries, 0 to 348882
Data columns (total 3 columns):
idpostulante              348883 non-null object
areas                     348883 non-null object
cantidad_postulaciones    348883 non-null int64
dtypes: int64(1), object(2)
memory usage: 8.0+ MB


Aqui vamos a agregar un feature que indique, por cada una de las top_areas que habiamos
definido cuando trabajamos con los avisos, que indique la cantidad de postulaciones que cada
usuarios realizo a esa area

In [56]:
for area in top_areas:
    lista_area = []
    for i in range(0,len(areas_por_postulante)):
        lista_areas_usuario = areas_por_postulante.loc[i,'areas']
        lista_area.append(lista_areas_usuario.count(area))
    
    areas_por_postulante['postulaciones_'+ area] = pd.Series(lista_area)

In [57]:
areas_por_postulante.head()

,idpostulante,areas,cantidad_postulaciones,postulaciones_Ventas,postulaciones_Comercial,postulaciones_Administración,postulaciones_Producción,postulaciones_Programación,postulaciones_Contabilidad,postulaciones_Tecnologia / Sistemas,...,postulaciones_Mantenimiento,postulaciones_Recursos Humanos,postulaciones_Logística,postulaciones_Gastronomia,postulaciones_Call Center,postulaciones_Soporte Técnico,postulaciones_Almacén / Depósito / Expedición,postulaciones_Oficios y Profesiones,postulaciones_Marketing,postulaciones_Recepcionista
0,0z5Dmrd,"[Recursos Humanos, Recursos Humanos]",2,0,0,0,0,0,0,0,...,0,2,0,0,0,0,0,0,0,0
1,0z5JW1r,"[Almacén / Depósito / Expedición, Comercial, D...",7,1,1,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
2,0z5VvGv,"[Ventas, Ventas, Atención al Cliente, Atención...",50,13,7,1,0,0,0,1,...,0,0,1,0,4,0,0,0,0,0
3,0zB01pE,"[Administracion de Seguros, Administracion de ...",3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0zB026d,"[Producción, Producción, Producción, Producció...",10,1,0,0,8,0,0,0,...,0,0,0,0,0,0,1,0,0,0


Agregamos al DF de usuarios estas columnas (features) que acabamos de calcular. 

In [58]:
usuarios = usuarios.merge(areas_por_postulante.drop(columns = 'areas'),
                          on = 'idpostulante',
                          how = 'left')
usuarios.head()

,idpostulante,edad,sexo_masculino,titulo_universitario,titulo_secundario,titulo_superior,cantidad_postulaciones,postulaciones_Ventas,postulaciones_Comercial,postulaciones_Administración,...,postulaciones_Mantenimiento,postulaciones_Recursos Humanos,postulaciones_Logística,postulaciones_Gastronomia,postulaciones_Call Center,postulaciones_Soporte Técnico,postulaciones_Almacén / Depósito / Expedición,postulaciones_Oficios y Profesiones,postulaciones_Marketing,postulaciones_Recepcionista
0,eo2p,37.0,True,True,True,True,6.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1d2B,42.0,True,False,False,False,4.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,EBO0,44.0,False,True,False,True,8.0,0.0,3.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,a6MKW,43.0,True,True,False,True,3.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,6MWd4,43.0,True,True,False,False,3.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [59]:
usuarios.isnull().sum()

idpostulante                                          0
edad                                                  0
sexo_masculino                                        0
titulo_universitario                                  0
titulo_secundario                                     0
titulo_superior                                       0
cantidad_postulaciones                           155524
postulaciones_Ventas                             155524
postulaciones_Comercial                          155524
postulaciones_Administración                     155524
postulaciones_Producción                         155524
postulaciones_Programación                       155524
postulaciones_Contabilidad                       155524
postulaciones_Tecnologia / Sistemas              155524
postulaciones_Atención al Cliente                155524
postulaciones_Mantenimiento                      155524
postulaciones_Recursos Humanos                   155524
postulaciones_Logística                         

Tenemos usuarios que no realizaron postulaciones, por eso poseen algunos features nulos, 
los completamos con ceros, que es lo que corresponde en este caso

In [60]:
usuarios.fillna(0, inplace = True)
usuarios.isnull().sum()

idpostulante                                     0
edad                                             0
sexo_masculino                                   0
titulo_universitario                             0
titulo_secundario                                0
titulo_superior                                  0
cantidad_postulaciones                           0
postulaciones_Ventas                             0
postulaciones_Comercial                          0
postulaciones_Administración                     0
postulaciones_Producción                         0
postulaciones_Programación                       0
postulaciones_Contabilidad                       0
postulaciones_Tecnologia / Sistemas              0
postulaciones_Atención al Cliente                0
postulaciones_Mantenimiento                      0
postulaciones_Recursos Humanos                   0
postulaciones_Logística                          0
postulaciones_Gastronomia                        0
postulaciones_Call Center      

### Volvemos a las postulaciones

In [61]:
postulaciones.head()

,idaviso,idpostulante,nombre_area
0,1112257047,NM5M,Atención al Cliente
1,1111920714,NM5M,Telemarketing
2,1112346945,NM5M,Telemarketing
3,1112345547,NM5M,Telemarketing
4,1112237522,5awk,Contabilidad


A cada postulacion le agregamos los features del aviso en cuestión

In [62]:
postulaciones.drop(columns = 'nombre_area', inplace = True)
postulaciones = postulaciones.merge(avisos, 
                                    on = 'idaviso',
                                    how = 'left')

postulaciones.head()

,idaviso,idpostulante,GBA,nombre_area,Full-time,Part-time,senior,junior,otro,Jefe,...,Mantenimiento,Recursos Humanos,Logística,Gastronomia,Call Center,Soporte Técnico,Almacén / Depósito / Expedición,Oficios y Profesiones,Marketing,Recepcionista
0,1112257047,NM5M,True,Atención al Cliente,False,True,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
1,1111920714,NM5M,True,Telemarketing,False,True,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,1112346945,NM5M,True,Telemarketing,True,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,1112345547,NM5M,True,Telemarketing,True,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,1112237522,5awk,True,Contabilidad,True,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [63]:
postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6603752 entries, 0 to 6603751
Data columns (total 29 columns):
idaviso                            int64
idpostulante                       object
GBA                                object
nombre_area                        object
Full-time                          object
Part-time                          object
senior                             object
junior                             object
otro                               object
Jefe                               object
Gerencia                           object
Ventas                             object
Comercial                          object
Administración                     object
Producción                         object
Programación                       object
Contabilidad                       object
Tecnologia / Sistemas              object
Atención al Cliente                object
Mantenimiento                      object
Recursos Humanos                   object
Logíst

Agregamos una columna que indique que se realizo una postulación que implica el par
(idpostulante, idaviso). Esto ahora parece no tener sentido, pero luego querremos tener
también pares que no impliquen una postulacion, y de esta forma podremos diferenciarlos

In [64]:
postulaciones['postulacion'] = True
postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6603752 entries, 0 to 6603751
Data columns (total 30 columns):
idaviso                            int64
idpostulante                       object
GBA                                object
nombre_area                        object
Full-time                          object
Part-time                          object
senior                             object
junior                             object
otro                               object
Jefe                               object
Gerencia                           object
Ventas                             object
Comercial                          object
Administración                     object
Producción                         object
Programación                       object
Contabilidad                       object
Tecnologia / Sistemas              object
Atención al Cliente                object
Mantenimiento                      object
Recursos Humanos                   object
Logíst

Ahora agregamos a la postulacion, al igual que agregamos los datos de los avisos, los datos
de los usuarios

In [65]:
postulaciones = postulaciones.merge(usuarios, on = 'idpostulante', how = 'left')
postulaciones.head()

,idaviso,idpostulante,GBA,nombre_area,Full-time,Part-time,senior,junior,otro,Jefe,...,postulaciones_Mantenimiento,postulaciones_Recursos Humanos,postulaciones_Logística,postulaciones_Gastronomia,postulaciones_Call Center,postulaciones_Soporte Técnico,postulaciones_Almacén / Depósito / Expedición,postulaciones_Oficios y Profesiones,postulaciones_Marketing,postulaciones_Recepcionista
0,1112257047,NM5M,True,Atención al Cliente,False,True,False,False,True,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1111920714,NM5M,True,Telemarketing,False,True,True,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1112346945,NM5M,True,Telemarketing,True,False,True,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1112345547,NM5M,True,Telemarketing,True,False,True,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1112237522,5awk,True,Contabilidad,True,False,True,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [66]:
postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6603752 entries, 0 to 6603751
Data columns (total 54 columns):
idaviso                                          int64
idpostulante                                     object
GBA                                              object
nombre_area                                      object
Full-time                                        object
Part-time                                        object
senior                                           object
junior                                           object
otro                                             object
Jefe                                             object
Gerencia                                         object
Ventas                                           object
Comercial                                        object
Administración                                   object
Producción                                       object
Programación                                    

Por alguna razon que desconozco, columnas que antes eran booleanas se transformaron en object
y aumento mucho el tamaño de los datos. Vamos a transformarlas nuevamente en booleanas

In [67]:
postulaciones.columns

Index(['idaviso', 'idpostulante', 'GBA', 'nombre_area', 'Full-time',
       'Part-time', 'senior', 'junior', 'otro', 'Jefe', 'Gerencia', 'Ventas',
       'Comercial', 'Administración', 'Producción', 'Programación',
       'Contabilidad', 'Tecnologia / Sistemas', 'Atención al Cliente',
       'Mantenimiento', 'Recursos Humanos', 'Logística', 'Gastronomia',
       'Call Center', 'Soporte Técnico', 'Almacén / Depósito / Expedición',
       'Oficios y Profesiones', 'Marketing', 'Recepcionista', 'postulacion',
       'edad', 'sexo_masculino', 'titulo_universitario', 'titulo_secundario',
       'titulo_superior', 'cantidad_postulaciones', 'postulaciones_Ventas',
       'postulaciones_Comercial', 'postulaciones_Administración',
       'postulaciones_Producción', 'postulaciones_Programación',
       'postulaciones_Contabilidad', 'postulaciones_Tecnologia / Sistemas',
       'postulaciones_Atención al Cliente', 'postulaciones_Mantenimiento',
       'postulaciones_Recursos Humanos', 'postulacion

In [68]:
columnas_booleanas = ['GBA','Full-time',
       'Part-time', 'senior', 'junior', 'otro', 'Jefe', 'Gerencia', 'Ventas',
       'Comercial', 'Administración', 'Producción', 'Programación',
       'Contabilidad', 'Tecnologia / Sistemas', 'Atención al Cliente',
       'Mantenimiento', 'Recursos Humanos', 'Logística', 'Gastronomia',
       'Call Center', 'Soporte Técnico', 'Almacén / Depósito / Expedición',
       'Oficios y Profesiones', 'Marketing', 'Recepcionista',
       'postulacion','sexo_masculino', 'titulo_universitario',
       'titulo_secundario', 'titulo_superior']

In [69]:
for columna in columnas_booleanas:
    postulaciones[columna] = postulaciones[columna].astype(bool)
    
postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6603752 entries, 0 to 6603751
Data columns (total 54 columns):
idaviso                                          int64
idpostulante                                     object
GBA                                              bool
nombre_area                                      object
Full-time                                        bool
Part-time                                        bool
senior                                           bool
junior                                           bool
otro                                             bool
Jefe                                             bool
Gerencia                                         bool
Ventas                                           bool
Comercial                                        bool
Administración                                   bool
Producción                                       bool
Programación                                     bool
Contabilidad      

Agregamos a las postulaciones la cantidad de visitas que cada usuario realizo al aviso
al que se postuló

In [70]:
postulaciones = postulaciones.merge(concatVistas,
                                    on = ['idaviso','idpostulante'],
                                    how = 'left')
postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6603752 entries, 0 to 6603751
Data columns (total 55 columns):
idaviso                                          int64
idpostulante                                     object
GBA                                              bool
nombre_area                                      object
Full-time                                        bool
Part-time                                        bool
senior                                           bool
junior                                           bool
otro                                             bool
Jefe                                             bool
Gerencia                                         bool
Ventas                                           bool
Comercial                                        bool
Administración                                   bool
Producción                                       bool
Programación                                     bool
Contabilidad      

In [71]:
postulaciones['visitas'].fillna(0,inplace = True)
postulaciones.head()

,idaviso,idpostulante,GBA,nombre_area,Full-time,Part-time,senior,junior,otro,Jefe,...,postulaciones_Recursos Humanos,postulaciones_Logística,postulaciones_Gastronomia,postulaciones_Call Center,postulaciones_Soporte Técnico,postulaciones_Almacén / Depósito / Expedición,postulaciones_Oficios y Profesiones,postulaciones_Marketing,postulaciones_Recepcionista,visitas
0,1112257047,NM5M,True,Atención al Cliente,False,True,False,False,True,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1111920714,NM5M,True,Telemarketing,False,True,True,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1112346945,NM5M,True,Telemarketing,True,False,True,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1112345547,NM5M,True,Telemarketing,True,False,True,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1112237522,5awk,True,Contabilidad,True,False,True,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [72]:
postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6603752 entries, 0 to 6603751
Data columns (total 55 columns):
idaviso                                          int64
idpostulante                                     object
GBA                                              bool
nombre_area                                      object
Full-time                                        bool
Part-time                                        bool
senior                                           bool
junior                                           bool
otro                                             bool
Jefe                                             bool
Gerencia                                         bool
Ventas                                           bool
Comercial                                        bool
Administración                                   bool
Producción                                       bool
Programación                                     bool
Contabilidad      

In [73]:
postulaciones.isnull().sum()

idaviso                                               0
idpostulante                                          0
GBA                                                   0
nombre_area                                      415823
Full-time                                             0
Part-time                                             0
senior                                                0
junior                                                0
otro                                                  0
Jefe                                                  0
Gerencia                                              0
Ventas                                                0
Comercial                                             0
Administración                                        0
Producción                                            0
Programación                                          0
Contabilidad                                          0
Tecnologia / Sistemas                           

Aqui ya tenemos un set de datos que nos serviría para entrenar a un algoritmo 
de Machine Learning. Este algoritmo debe predecir si un postulante se postuló o no a un 
determinado aviso basandose en los features que definimos. Se podria decir que se trata
de un problema de clasificación binaria, donde las categorias posibles de clasificación son
'postulacion' o 'no_postulacion'. 
Bajo esta caracterización del problema, ahora nuestro set de entrenamiento resulta incompleto,
ya que solo tenemos datos de una categoría ('postulacion' o columna 'postulacion' = True).
Por lo tanto debemos generar datos que correspondan a pares de usuarios y avisos cuya 
categoría corresponda a 'no_postulacion'.

## Generando No_Postulaciones

Tomo al azar una cantidad de usuarios y avisos. Los uno indice a indice, y luego le doy al 
DF el mismo formato que el DF de postulaciones

In [74]:
no_postulaciones = usuarios.sample(10000)
for i in range(1,400):
    no_postulaciones = pd.concat([no_postulaciones, usuarios.sample(25000)])

In [75]:
avisos_no_postulaciones = avisos.sample(10000)
for i in range(1,400):
    avisos_no_postulaciones = pd.concat([avisos_no_postulaciones,avisos.sample(25000)])

In [76]:
no_postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9985000 entries, 44030 to 7697
Data columns (total 25 columns):
idpostulante                                     object
edad                                             float64
sexo_masculino                                   bool
titulo_universitario                             bool
titulo_secundario                                bool
titulo_superior                                  bool
cantidad_postulaciones                           float64
postulaciones_Ventas                             float64
postulaciones_Comercial                          float64
postulaciones_Administración                     float64
postulaciones_Producción                         float64
postulaciones_Programación                       float64
postulaciones_Contabilidad                       float64
postulaciones_Tecnologia / Sistemas              float64
postulaciones_Atención al Cliente                float64
postulaciones_Mantenimiento                 

In [77]:
no_postulaciones.head()

,idpostulante,edad,sexo_masculino,titulo_universitario,titulo_secundario,titulo_superior,cantidad_postulaciones,postulaciones_Ventas,postulaciones_Comercial,postulaciones_Administración,...,postulaciones_Mantenimiento,postulaciones_Recursos Humanos,postulaciones_Logística,postulaciones_Gastronomia,postulaciones_Call Center,postulaciones_Soporte Técnico,postulaciones_Almacén / Depósito / Expedición,postulaciones_Oficios y Profesiones,postulaciones_Marketing,postulaciones_Recepcionista
44030,ak4zv3N,23.0,True,False,True,False,20.0,2.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
121562,0zrO40a,26.0,True,False,True,False,13.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
173939,Rz9mAkO,44.0,False,False,False,False,7.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
445073,A361lpm,24.0,False,False,False,False,4.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19799,akbNYk9,25.0,True,False,True,False,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [78]:
no_postulaciones.reset_index(inplace = True)
no_postulaciones.drop(columns = 'index', inplace = True)
no_postulaciones.head()

,idpostulante,edad,sexo_masculino,titulo_universitario,titulo_secundario,titulo_superior,cantidad_postulaciones,postulaciones_Ventas,postulaciones_Comercial,postulaciones_Administración,...,postulaciones_Mantenimiento,postulaciones_Recursos Humanos,postulaciones_Logística,postulaciones_Gastronomia,postulaciones_Call Center,postulaciones_Soporte Técnico,postulaciones_Almacén / Depósito / Expedición,postulaciones_Oficios y Profesiones,postulaciones_Marketing,postulaciones_Recepcionista
0,ak4zv3N,23.0,True,False,True,False,20.0,2.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0zrO40a,26.0,True,False,True,False,13.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Rz9mAkO,44.0,False,False,False,False,7.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,A361lpm,24.0,False,False,False,False,4.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,akbNYk9,25.0,True,False,True,False,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [79]:
avisos_no_postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9985000 entries, 5574 to 12002
Data columns (total 28 columns):
idaviso                            int64
GBA                                bool
nombre_area                        object
Full-time                          bool
Part-time                          bool
senior                             bool
junior                             bool
otro                               bool
Jefe                               bool
Gerencia                           bool
Ventas                             bool
Comercial                          bool
Administración                     bool
Producción                         bool
Programación                       bool
Contabilidad                       bool
Tecnologia / Sistemas              bool
Atención al Cliente                bool
Mantenimiento                      bool
Recursos Humanos                   bool
Logística                          bool
Gastronomia                        bool
Cal

In [80]:
avisos_no_postulaciones.head()

,idaviso,GBA,nombre_area,Full-time,Part-time,senior,junior,otro,Jefe,Gerencia,...,Mantenimiento,Recursos Humanos,Logística,Gastronomia,Call Center,Soporte Técnico,Almacén / Depósito / Expedición,Oficios y Profesiones,Marketing,Recepcionista
5574,1112207834,True,Recepcionista,True,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
13020,1112324289,True,Impuestos,True,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
11919,1112514047,True,Finanzas,True,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
10607,1112502341,True,Ventas,True,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
71,1112342806,True,Planeamiento comercial,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [81]:
avisos_no_postulaciones.reset_index(inplace = True)
avisos_no_postulaciones.drop(columns = 'index', inplace = True)
avisos_no_postulaciones.head()

,idaviso,GBA,nombre_area,Full-time,Part-time,senior,junior,otro,Jefe,Gerencia,...,Mantenimiento,Recursos Humanos,Logística,Gastronomia,Call Center,Soporte Técnico,Almacén / Depósito / Expedición,Oficios y Profesiones,Marketing,Recepcionista
0,1112207834,True,Recepcionista,True,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
1,1112324289,True,Impuestos,True,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,1112514047,True,Finanzas,True,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,1112502341,True,Ventas,True,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,1112342806,True,Planeamiento comercial,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [82]:
avisos_no_postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9985000 entries, 0 to 9984999
Data columns (total 28 columns):
idaviso                            int64
GBA                                bool
nombre_area                        object
Full-time                          bool
Part-time                          bool
senior                             bool
junior                             bool
otro                               bool
Jefe                               bool
Gerencia                           bool
Ventas                             bool
Comercial                          bool
Administración                     bool
Producción                         bool
Programación                       bool
Contabilidad                       bool
Tecnologia / Sistemas              bool
Atención al Cliente                bool
Mantenimiento                      bool
Recursos Humanos                   bool
Logística                          bool
Gastronomia                        bool
Call

In [83]:
no_postulaciones = no_postulaciones.join(avisos_no_postulaciones)
no_postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9985000 entries, 0 to 9984999
Data columns (total 53 columns):
idpostulante                                     object
edad                                             float64
sexo_masculino                                   bool
titulo_universitario                             bool
titulo_secundario                                bool
titulo_superior                                  bool
cantidad_postulaciones                           float64
postulaciones_Ventas                             float64
postulaciones_Comercial                          float64
postulaciones_Administración                     float64
postulaciones_Producción                         float64
postulaciones_Programación                       float64
postulaciones_Contabilidad                       float64
postulaciones_Tecnologia / Sistemas              float64
postulaciones_Atención al Cliente                float64
postulaciones_Mantenimiento                  

In [84]:
no_postulaciones['postulacion'] = False
no_postulaciones.head()

,idpostulante,edad,sexo_masculino,titulo_universitario,titulo_secundario,titulo_superior,cantidad_postulaciones,postulaciones_Ventas,postulaciones_Comercial,postulaciones_Administración,...,Recursos Humanos,Logística,Gastronomia,Call Center,Soporte Técnico,Almacén / Depósito / Expedición,Oficios y Profesiones,Marketing,Recepcionista,postulacion
0,ak4zv3N,23.0,True,False,True,False,20.0,2.0,0.0,0.0,...,False,False,False,False,False,False,False,False,True,False
1,0zrO40a,26.0,True,False,True,False,13.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
2,Rz9mAkO,44.0,False,False,False,False,7.0,1.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
3,A361lpm,24.0,False,False,False,False,4.0,0.0,1.0,1.0,...,False,False,False,False,False,False,False,False,False,False
4,akbNYk9,25.0,True,False,True,False,2.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False


In [85]:
no_postulaciones = no_postulaciones.merge(concatVistas,
                                          on = ['idaviso','idpostulante'],
                                          how = 'left')
no_postulaciones.head()

,idpostulante,edad,sexo_masculino,titulo_universitario,titulo_secundario,titulo_superior,cantidad_postulaciones,postulaciones_Ventas,postulaciones_Comercial,postulaciones_Administración,...,Logística,Gastronomia,Call Center,Soporte Técnico,Almacén / Depósito / Expedición,Oficios y Profesiones,Marketing,Recepcionista,postulacion,visitas
0,ak4zv3N,23.0,True,False,True,False,20.0,2.0,0.0,0.0,...,False,False,False,False,False,False,False,True,False,NaN
1,0zrO40a,26.0,True,False,True,False,13.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,NaN
2,Rz9mAkO,44.0,False,False,False,False,7.0,1.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,NaN
3,A361lpm,24.0,False,False,False,False,4.0,0.0,1.0,1.0,...,False,False,False,False,False,False,False,False,False,NaN
4,akbNYk9,25.0,True,False,True,False,2.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,NaN


In [86]:
no_postulaciones['visitas'].fillna(0, inplace = True)
no_postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9985000 entries, 0 to 9984999
Data columns (total 55 columns):
idpostulante                                     object
edad                                             float64
sexo_masculino                                   bool
titulo_universitario                             bool
titulo_secundario                                bool
titulo_superior                                  bool
cantidad_postulaciones                           float64
postulaciones_Ventas                             float64
postulaciones_Comercial                          float64
postulaciones_Administración                     float64
postulaciones_Producción                         float64
postulaciones_Programación                       float64
postulaciones_Contabilidad                       float64
postulaciones_Tecnologia / Sistemas              float64
postulaciones_Atención al Cliente                float64
postulaciones_Mantenimiento                  

In [87]:
no_postulaciones = no_postulaciones[postulaciones.columns]
no_postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9985000 entries, 0 to 9984999
Data columns (total 55 columns):
idaviso                                          int64
idpostulante                                     object
GBA                                              bool
nombre_area                                      object
Full-time                                        bool
Part-time                                        bool
senior                                           bool
junior                                           bool
otro                                             bool
Jefe                                             bool
Gerencia                                         bool
Ventas                                           bool
Comercial                                        bool
Administración                                   bool
Producción                                       bool
Programación                                     bool
Contabilidad      

Las no postulaciones ya tienen el mismo formato que las postulaciones, con la columna que 
corresponde a las postulaciones en False.
Vamos a concatenar postulaciones y no_postulaciones en el DF de postulaciones, para tener
un solo set de datos compacto

In [88]:
postulaciones = pd.concat([postulaciones,no_postulaciones])

In [89]:
postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16588752 entries, 0 to 9984999
Data columns (total 55 columns):
idaviso                                          int64
idpostulante                                     object
GBA                                              bool
nombre_area                                      object
Full-time                                        bool
Part-time                                        bool
senior                                           bool
junior                                           bool
otro                                             bool
Jefe                                             bool
Gerencia                                         bool
Ventas                                           bool
Comercial                                        bool
Administración                                   bool
Producción                                       bool
Programación                                     bool
Contabilidad     

Es posible que cuando generé las no_postulaciones de forma random, se hayan generado pares
(idpostulante, idaviso) que de hecho correspondian a postulaciones. Solucionamos esto 
eliminando los pares duplicados y quedandonos con la primera aparición de cada par.

In [90]:
postulaciones.drop_duplicates(subset = ['idaviso','idpostulante'], keep = 'first',inplace = True)
postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16580030 entries, 0 to 9984999
Data columns (total 55 columns):
idaviso                                          int64
idpostulante                                     object
GBA                                              bool
nombre_area                                      object
Full-time                                        bool
Part-time                                        bool
senior                                           bool
junior                                           bool
otro                                             bool
Jefe                                             bool
Gerencia                                         bool
Ventas                                           bool
Comercial                                        bool
Administración                                   bool
Producción                                       bool
Programación                                     bool
Contabilidad     

Voy a eliminar del set de datos aquellas filas que correspondan con usuarios inactivos, ya
que una no_postulacion por parte de ellos a un determinado aviso no estoy seguro de que me 
indique que ambos no sean compatibles, puede ser simplemente que el usuario ni siquiera 
busco avisos que le interesen

In [91]:
postulaciones = postulaciones.loc[postulaciones['cantidad_postulaciones'] != 0]

In [92]:
len(postulaciones)

13501638

In [93]:
postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13501638 entries, 0 to 9984999
Data columns (total 55 columns):
idaviso                                          int64
idpostulante                                     object
GBA                                              bool
nombre_area                                      object
Full-time                                        bool
Part-time                                        bool
senior                                           bool
junior                                           bool
otro                                             bool
Jefe                                             bool
Gerencia                                         bool
Ventas                                           bool
Comercial                                        bool
Administración                                   bool
Producción                                       bool
Programación                                     bool
Contabilidad     

Ahora si, ya tenemos nuestro set de datos listo para entrenar un algoritmo de Machine Learning

# Importando el set de datos corresponiente al Test final

Importamos el csv test_final. Que tiene formato (id,idaviso,idpostulante).
Debemos darle el mismo formato que tiene el df postulaciones, agregando los features
correspondientes. Es claro que este df tendrá una columna menos, la de postulaciones, ya que 
esa es la columna que el algoritmo de ML deberá predecir.

In [94]:
test_final = pd.read_csv('data/test_final_100k.csv')

In [95]:
test_final = test_final.merge(usuarios, on='idpostulante', how = 'left')
test_final.head()

,id,idaviso,idpostulante,edad,sexo_masculino,titulo_universitario,titulo_secundario,titulo_superior,cantidad_postulaciones,postulaciones_Ventas,...,postulaciones_Mantenimiento,postulaciones_Recursos Humanos,postulaciones_Logística,postulaciones_Gastronomia,postulaciones_Call Center,postulaciones_Soporte Técnico,postulaciones_Almacén / Depósito / Expedición,postulaciones_Oficios y Profesiones,postulaciones_Marketing,postulaciones_Recepcionista
0,0,739260,6M9ZQR,42.0,False,True,False,True,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1,739260,6v1xdL,30.0,True,False,False,False,68.0,4.0,...,0.0,0.0,1.0,3.0,4.0,0.0,0.0,0.0,0.0,1.0
2,2,739260,ezRKm9,36.0,False,True,False,False,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,758580,1Q35ej,68.0,True,True,False,True,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,758580,EAN4J6,32.0,False,False,False,False,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [96]:
test_final = test_final.merge(avisos,
                            on = 'idaviso',
                            how = 'left')
test_final.head()

,id,idaviso,idpostulante,edad,sexo_masculino,titulo_universitario,titulo_secundario,titulo_superior,cantidad_postulaciones,postulaciones_Ventas,...,Mantenimiento,Recursos Humanos,Logística,Gastronomia,Call Center,Soporte Técnico,Almacén / Depósito / Expedición,Oficios y Profesiones,Marketing,Recepcionista
0,0,739260,6M9ZQR,42.0,False,True,False,True,2.0,0.0,...,False,False,False,False,False,False,False,False,False,False
1,1,739260,6v1xdL,30.0,True,False,False,False,68.0,4.0,...,False,False,False,False,False,False,False,False,False,False
2,2,739260,ezRKm9,36.0,False,True,False,False,1.0,0.0,...,False,False,False,False,False,False,False,False,False,False
3,3,758580,1Q35ej,68.0,True,True,False,True,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
4,4,758580,EAN4J6,32.0,False,False,False,False,1.0,0.0,...,False,False,False,False,False,False,False,False,False,False


In [97]:
test_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 54 columns):
id                                               100000 non-null int64
idaviso                                          100000 non-null int64
idpostulante                                     100000 non-null object
edad                                             100000 non-null float64
sexo_masculino                                   100000 non-null bool
titulo_universitario                             100000 non-null bool
titulo_secundario                                100000 non-null bool
titulo_superior                                  100000 non-null bool
cantidad_postulaciones                           100000 non-null float64
postulaciones_Ventas                             100000 non-null float64
postulaciones_Comercial                          100000 non-null float64
postulaciones_Administración                     100000 non-null float64
postulaciones_Producción       

In [98]:
test_final = test_final.merge(concatVistas, on = ['idaviso','idpostulante'], how = 'left')
test_final.head()

,id,idaviso,idpostulante,edad,sexo_masculino,titulo_universitario,titulo_secundario,titulo_superior,cantidad_postulaciones,postulaciones_Ventas,...,Recursos Humanos,Logística,Gastronomia,Call Center,Soporte Técnico,Almacén / Depósito / Expedición,Oficios y Profesiones,Marketing,Recepcionista,visitas
0,0,739260,6M9ZQR,42.0,False,True,False,True,2.0,0.0,...,False,False,False,False,False,False,False,False,False,NaN
1,1,739260,6v1xdL,30.0,True,False,False,False,68.0,4.0,...,False,False,False,False,False,False,False,False,False,NaN
2,2,739260,ezRKm9,36.0,False,True,False,False,1.0,0.0,...,False,False,False,False,False,False,False,False,False,NaN
3,3,758580,1Q35ej,68.0,True,True,False,True,0.0,0.0,...,False,False,False,False,False,False,False,False,False,NaN
4,4,758580,EAN4J6,32.0,False,False,False,False,1.0,0.0,...,False,False,False,False,False,False,False,False,False,NaN


In [99]:
test_final['visitas'].fillna(0, inplace = True)
test_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 55 columns):
id                                               100000 non-null int64
idaviso                                          100000 non-null int64
idpostulante                                     100000 non-null object
edad                                             100000 non-null float64
sexo_masculino                                   100000 non-null bool
titulo_universitario                             100000 non-null bool
titulo_secundario                                100000 non-null bool
titulo_superior                                  100000 non-null bool
cantidad_postulaciones                           100000 non-null float64
postulaciones_Ventas                             100000 non-null float64
postulaciones_Comercial                          100000 non-null float64
postulaciones_Administración                     100000 non-null float64
postulaciones_Producción       

Ahora tenemos todo listo para entrenar nuestro algoritmo de ML

# Empieza perceptron


La columna 'nombre_area' no nos va a servir para este algoritmo, la vamos a dejar de lado.
Lo mismo con los ids, que no son features en sí

In [100]:
datos = postulaciones.drop(columns = ['idpostulante','idaviso','postulacion','nombre_area'])
categorias = postulaciones['postulacion']

In [101]:
datos.head()

,GBA,Full-time,Part-time,senior,junior,otro,Jefe,Gerencia,Ventas,Comercial,...,postulaciones_Recursos Humanos,postulaciones_Logística,postulaciones_Gastronomia,postulaciones_Call Center,postulaciones_Soporte Técnico,postulaciones_Almacén / Depósito / Expedición,postulaciones_Oficios y Profesiones,postulaciones_Marketing,postulaciones_Recepcionista,visitas
0,True,False,True,False,False,True,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,True,False,True,True,False,False,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,True,True,False,True,False,False,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,True,True,False,True,False,False,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,True,True,False,True,False,False,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [102]:
test_size = 0.33
random_state = 0

In [103]:
datos_train, datos_test, cat_train, cat_test = train_test_split(datos,
                                                               categorias,
                                                               test_size = test_size,
                                                               random_state = random_state)

In [104]:
sc = StandardScaler()
sc.fit(datos_train)

datos_train_std = sc.transform(datos_train)
datos_test_std = sc.transform(datos_test)

In [105]:
len(datos_train_std)

9046097

In [106]:
#data_train = datos_train_std[:,0:5]
#data_test = datos_test_std[:,0:5]
n_iter=40
eta0 = 0.1

In [107]:
#ppn2 con todos los features
ppn2 = Perceptron(max_iter = n_iter,
                eta0 = eta0,
                random_state = random_state)

ppn2.fit(datos_train_std,cat_train)



Perceptron(alpha=0.0001, class_weight=None, eta0=0.1, fit_intercept=True,
      max_iter=40, n_iter=None, n_jobs=1, penalty=None, random_state=0,
      shuffle=True, tol=None, verbose=0, warm_start=False)

In [108]:
cat_pred = ppn2.predict(datos_test_std)

print('accuracy: {0:.2f}%'.format(accuracy_score(cat_test,cat_pred)*100))

accuracy: 71.16%


In [109]:
final_test_data = test_final[datos.columns]

In [110]:
final_test_data.head()

,GBA,Full-time,Part-time,senior,junior,otro,Jefe,Gerencia,Ventas,Comercial,...,postulaciones_Recursos Humanos,postulaciones_Logística,postulaciones_Gastronomia,postulaciones_Call Center,postulaciones_Soporte Técnico,postulaciones_Almacén / Depósito / Expedición,postulaciones_Oficios y Profesiones,postulaciones_Marketing,postulaciones_Recepcionista,visitas
0,False,True,False,False,False,False,True,False,False,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,False,True,False,False,False,False,True,False,False,True,...,0.0,1.0,3.0,4.0,0.0,0.0,0.0,0.0,1.0,0.0
2,False,True,False,False,False,False,True,False,False,True,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,False,True,False,False,False,True,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,False,True,False,False,False,True,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [111]:
final_test_data = sc.transform(final_test_data)

In [112]:
prediccion = ppn2.predict(final_test_data)
prediccion = prediccion.astype(int)

In [113]:
test_final['sepostulo'] = prediccion
entregable = test_final[['id','sepostulo']]

In [114]:
entregable.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 2 columns):
id           100000 non-null int64
sepostulo    100000 non-null int64
dtypes: int64(2)
memory usage: 2.3 MB


In [115]:
entregable.index = entregable['id']
entregable.drop(columns = 'id', inplace = True)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [116]:
#hasta ahora mi mejor submit
entregable2 = pd.read_csv('submits/submit_9.csv')

In [117]:
print('accuracy: {0:.2f}%'.format(accuracy_score(entregable['sepostulo'],entregable2['sepostulo'])*100))

accuracy: 80.83%


In [118]:
entregable.to_csv('submits/submit_21.csv')

# Empieza KNN

In [ ]:
datos.columns

In [ ]:
datos2 = datos.drop(columns = ['senior','junior','otro','Jefe','Gerencia','titulo_superior','Part-time'])

In [ ]:
datos_train, datos_test, cat_train, cat_test = train_test_split(datos2,
                                                               categorias,
                                                               test_size = test_size,
                                                               random_state = random_state)

In [ ]:
sc = StandardScaler()
sc.fit(datos_train)

datos_train_std = sc.transform(datos_train)
datos_test_std = sc.transform(datos_test)

In [ ]:
n_iter=60
eta0 = 0.1

In [ ]:
#ppn2 con todos los features
ppn2 = Perceptron(n_iter = n_iter,
                eta0 = eta0,
                random_state = random_state)

ppn2.fit(datos_train_std,cat_train)

In [ ]:
cat_pred = ppn2.predict(datos_test_std)

print('accuracy: {0:.2f}%'.format(accuracy_score(cat_test,cat_pred)*100))

In [ ]:
final_test_data = test_final[datos2.columns]

In [ ]:
final_test_data.head()

In [ ]:
final_test_data = sc.transform(final_test_data)

In [ ]:
prediccion = ppn2.predict(final_test_data)
prediccion = prediccion.astype(int)
test_final['sepostulo'] = prediccion
entregable = test_final[['id','sepostulo']]

In [ ]:
#hasta ahora mi mejor submit
entregable2 = pd.read_csv('submits/submit_9.csv')

In [ ]:
print('accuracy: {0:.2f}%'.format(accuracy_score(entregable['sepostulo'],
                                                 entregable2['sepostulo'])*100))